In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
    !pip install synthetic-data-kit==0.0.3
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1
    !pip install synthetic-data-kit==0.0.3
    !pip install transformers==4.51.3


In [3]:
from unsloth.dataprep import SyntheticDataKit

generator = SyntheticDataKit.from_pretrained(
    # Choose any model from https://huggingface.co/unsloth
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Longer sequence lengths will be slower!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/herooooooooo/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-07 16:25:35 [__init__.py:243] Automatically detected platform cuda.


2025-06-07 16:25:36,402	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Unsloth: Using dtype = torch.bfloat16 for vLLM.
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 88.54%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 15.6 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.83 GB. Also swap space = 2 GB.
vLLM STDOUT: INFO 06-07 16:25:44 [__init__.py:243] Automatically detected platform cuda.
vLLM STDOUT: INFO 06-07 16:25:46 [__init__.py:31] Available plugins for group vllm.general_plugins:
vLLM STDOUT: INFO 06-07 16:25:46 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
vLLM STDOUT: INFO 06-07 16:25:46 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
vLLM STDOUT: INFO 06-07 16:25:47 [api_server.py:1289] vLLM API server version 0.9.0.1
vLLM STDOUT: INFO 06-07 16:25:48 [cli_args.py:300] n

In [5]:
generator.prepare_qa_generation(
    output_folder = "data", # Output location of synthetic data
    temperature = 0.7, # Higher temp makes more diverse datases
    top_p = 0.95,
    overlap = 64, # Overlap portion during chunking
    max_generation_tokens = 512, # Can increase for longer QA pairs
)

In [7]:
!synthetic-data-kit system-check

 VLLM server is running at http://localhost:8000/v1
Available models: {'object': 'list', 'data': [{'id': 
'unsloth/Llama-3.2-3B-Instruct', 'object': 'model', 'created': 1749300234, 
'owned_by': 'vllm', 'root': 'unsloth/Llama-3.2-3B-Instruct', 'parent': None, 
'max_model_len': 2048, 'permission': [{'id': 
'modelperm-d3551765d5494e4386b5ba99dbfe397c', 'object': 'model_permission', 
'created': 1749300234, 'allow_create_engine': False, 'allow_sampling': True, 
'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 
'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': 
False}]}]}
⠋ Checking VLLM server at http://localhost:8000/v1...


In [ ]:
# Byte Latent Transformer: Patches Scale Better Than Tokens paper in HTML format
!synthetic-data-kit \
    -c synthetic_data_kit_config.yaml \
    ingest "https://arxiv.org/html/2412.09871v1"

# Truncate document
filenames = generator.chunk_data("data/output/arxiv_org.txt")
print(len(filenames), filenames[:3])

⠦ Processing https://arxiv.org/html/2412.09871v1.....
 Text successfully extracted to data/output/arxiv_org.txt
37 ['data/output/arxiv_org_0.txt', 'data/output/arxiv_org_1.txt', 'data/output/arxiv_org_2.txt']


In [11]:
import time
# Process 3 chunks for now -> can increase but slower!
for filename in filenames[:3]:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create {filename} \
        --num-pairs 25 \
        --type "qa"
    time.sleep(2) # Sleep some time to leave some room for processing

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing 1 chunks to generate QA pairs...output/arxiv_org_0.txt.....
Batch processing complete.ontent from data/output/arxiv_org_0.txt...
Generated 9 QA pairs totalnt from data/output/arxiv_org_0.txt...
Saving result to data/generated/arxiv_org_0_qa_pairs.json.txt...
Successfully wrote test file to data/generated/test_write.json..
Successfully wrote result to data/generated/arxiv_org_0_qa_pairs.json
⠋ Generating qa content from data/output/arxiv_org_0.txt...
 Content saved to data/generated/arxiv_org_0_qa_pairs.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing 1 chunks to generate QA pairs...output/arxiv_org_1.txt.....
Batch processing complete.ontent from data/output/arxiv_org_1.txt...
Generated 13 QA pairs totalt from data/output/arxiv_org_1.txt...
Saving result to data/generated/arxiv_org_1_qa_pairs.json.txt...
Successfully wrote test file to data/generated/test_write.json..
Successfully wrote result to data/generated/arxiv_org_1_qa_pairs.json
⠹ Generating qa content from data/output/arxiv_org_1.txt...
 Content saved to data/generated/arxiv_org_1_qa_pairs.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing 1 chunks to generate QA pairs...output/arxiv_org_2.txt.....
Batch processing complete.ontent from data/output/arxiv_org_2.txt...
Generated 11 QA pairs totalt from data/output/arxiv_org_2.txt...
Saving result to data/generated/arxiv_org_2_qa_pairs.json.txt...
Successfully wrote test file to data/generated/test_write.json..
Successfully wrote result to data/generated/arxiv_org_2_qa_pairs.json
⠸ Generating qa content from data/output/arxiv_org_2.txt...
 Content saved to data/generated/arxiv_org_2_qa_pairs.json


In [12]:
!synthetic-data-kit \
    -c synthetic_data_kit_config.yaml \
    curate --threshold 0.0 \
    "data/generated/arxiv_org_0_qa_pairs.json"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing 2 batches of QA pairs...
Batch processing complete.t from data/generated/arxiv_org_0_qa_pairs.json...
Rated 9 QA pairscontent from data/generated/arxiv_org_0_qa_pairs.json...
Retained 9 pairs (threshold: 0.0)/generated/arxiv_org_0_qa_pairs.json...
Average score: 8.3ntent from data/generated/arxiv_org_0_qa_pairs.json...
⠴ Cleaning content from data/generated/arxiv_org_0_qa_pairs.json...
 Cleaned content saved to data/cleaned/arxiv_org_0_qa_pairs_cleaned.json


In [13]:
qa_pairs_filenames = [
    f"data/generated/arxiv_org_{i}_qa_pairs.json"
    for i in range(len(filenames[:3]))
]
for filename in qa_pairs_filenames:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


⠋ Converting data/generated/arxiv_org_0_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_0_qa_pairs_ft.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


⠋ Converting data/generated/arxiv_org_1_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_1_qa_pairs_ft.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


⠋ Converting data/generated/arxiv_org_2_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_2_qa_pairs_ft.json


In [14]:
from datasets import Dataset
import pandas as pd
final_filenames = [
    f"data/final/arxiv_org_{i}_qa_pairs_ft.json"
    for i in range(len(filenames[:3]))
]
conversations = pd.concat([
    pd.read_json(name) for name in final_filenames
]).reset_index(drop = True)

dataset = Dataset.from_pandas(conversations)

In [15]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the main innovation of the Byte Latent Transformer (BLT) architecture?',
   'role': 'user'},
  {'content': 'BLT encodes bytes into dynamically sized patches, which serve as the primary units of computation.',
   'role': 'assistant'}]}

In [16]:
generator.cleanup()

Attempting to terminate the VLLM server gracefully...
Server terminated gracefully.


In [17]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
    # Qwen3 new models
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # Other very popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.52.4. vLLM: 0.9.0.1.
   \\   /|    NVIDIA GeForce RTX 4090 Laptop GPU. Num GPUs = 1. Max memory: 15.596 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.2` format for conversation style finetunes. The chat template renders conversations like below: (Cutting Knowledge Date is by default there!)

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 01 May 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is 1+1?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

2<|eot_id|>
```

In [19]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

# Get our previous dataset and format it:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 33/33 [00:00<00:00, 2522.87 examples/s]


In [20]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the main innovation of the Byte Latent Transformer (BLT) architecture?',
   'role': 'user'},
  {'content': 'BLT encodes bytes into dynamically sized patches, which serve as the primary units of computation.',
   'role': 'assistant'}],
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Jun 2025\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the main innovation of the Byte Latent Transformer (BLT) architecture?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nBLT encodes bytes into dynamically sized patches, which serve as the primary units of computation.<|eot_id|>'}

In [21]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=32): 100%|██████████| 33/33 [00:03<00:00,  8.28 examples/s]


In [22]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090 Laptop GPU. Max memory = 15.596 GB.
3.441 GB of memory reserved.


In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33 | Num Epochs = 12 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,4.531500
2,4.660700
3,4.772200
4,4.385400
5,4.152700
6,3.699600
7,3.025400
8,2.874200
9,2.722400
10,2.865200


In [24]:
messages = [
    {"role": "user", "content": "What is the Byte Latent Transformer?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


BLT is a lightweight architecture that combines the efficiency of byte tokenization with the scaling capabilities of transformer models.<|eot_id|>


In [25]:
messages = [
    {"role": "user", "content": "What are some benefits of the BLT?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

The BLT provides improvements in both training and inference efficiency while maintaining a comparable number of floating-point operations.<|eot_id|>


In [ ]:
model.push_to_hub("herooooooooo/llama_finetuned_lora", token = "...") # Online saving
tokenizer.push_to_hub("herooooooooo/llama_finetuned_lora", token = "...") # Online saving

100%|██████████| 1/1 [00:02<00:00,  2.56s/it]


Saved model to https://huggingface.co/herooooooooo/llama_finetuned_lora


100%|██████████| 1/1 [00:01<00:00,  1.74s/it]
